# Load Mastodon tracking data on branch level into Napari to use with Napari Clusters Plotter

In [1]:
from pathlib import Path
import napari
import pandas as pd
from skimage.io import imread

## Prerequisites

Before running this notebook, the following steps need to be performed:
* Due to a bug in Napari clusters plotter handling tracking data, it may be required to install the latest version of Napari clusters plotter:
```
mamba activate devbio-napari
pip install git+https://github.com/BiAPoL/napari-clusters-plotter.git
```
* Compute features on BranchSpot Level in Mastodon.
  * Open "Compute Features View"
  * Select all available Branch Spot features > Compute
  * Open "Table View"
  * Select BranchSpot 
  * File > Export to CSV
* Export label image from Mastodon
  * Plugins > Exports > Export label image using ellipsoids
  * Select "Branch Spot ID" as label id
  * Select a frame reduction rate of 1
    * Higher frame reduction rates are possible and may speed up analyses in napari clusters plotter
    * If a frame reduction rate higher than 1 is used, a version of the intensity image would need to be created (e.g. with Fiji/BigStitcher) with the same frame reduction rate or no intensity image can be loaded into napari 

## Read tables from Mastodon

#### Set path to table
Enter the path to the tables here. Within that folder, there should be one table:
* BranchSpot.csv

In [2]:
tables_folder_path = ''

#### Read table from path
Rows with NaN values are removed.

In [3]:
branch_spot_table_path = Path(tables_folder_path + '/BranchSpot.csv')
branch_spot_table = pd.read_csv(branch_spot_table_path, skiprows=[1,2], encoding='utf-8')
# remove rows with NaN values
branch_spot_table = branch_spot_table.dropna()

#### Optionally print head of branch spot table to check if everything is ok

In [4]:
branch_spot_table.head(2)

,Label,ID,Branch Average Movement,Branch Movement Direction,Branch Movement Direction.1,Branch Movement Direction.2,Branch N Successors and Predecessors,Branch N Successors and Predecessors.1,Branch N leaves,Branch N spots,...,Detection.6,Division,Division.1,Division.2,Proliferator,Proliferator.1,Proliferator.2,Status,Tracking,Tracking.1
0,50644,0,0.845989,0.888329,0.458005,-0.033209,2,0,2,283,...,0,0,0,0,1,0,0,1,1,0
1,28749,1,0.785019,-0.840221,0.491131,-0.229825,2,0,2,176,...,0,0,0,0,1,0,0,1,1,0


## Change table to match napari-clusters-plotter standards
Mastodon 'label' column needs to be removed from the table and replaced with a new column 'label' that contains the branch spot ID.
The branch spot ID is the Mastodon ID + 1. This is necessary, since the ellipsoid label image starts with 1, while the Mastodon ID starts at 0.

In [5]:
# Remove Label column from Mastodon tables
branch_spot_table = branch_spot_table.drop(columns=['Label'])

# Add a new label column representing the branch spot ID
branch_spot_table['label'] = branch_spot_table['ID'].astype(int) + 1 # Turning branch spot ids into labels, NB: + 1 needs to be added, since the ids are counted one based in the Mastodon


### Currently available Branch spot features:
* label
* Branch Average Movement
* Branch Movement Direction (X)
* Branch Movement Direction.1 (Y)
* Branch Movement Direction.2 (Z)
* Branch N Successors and Predecessors (successors)
* Branch N Successors and Predecessors.1 predecessors (predecessors)
* Branch Sinuosity
* Branch duration and displacement (displacement)
* Branch duration and displacement.1 (duration)
* Branch Relative Movement (x-component)
* Branch Relative Movement.1 (y-component)
* Branch Relative Movement.2 (z-component)
* Branch Relative Movement.3 (average speed relative to 5 nearest neighbors)
* Cell Division Frequency

### Optional: create a cell fate column 
This is only useful, if the cell fate has been annotated in Mastodon.

In [6]:
# Define a function to determine the combined value
def cell_fate_values_to_label(row):
    if row['cell_fate']:
        return 1
    elif row['cell_fate.1']:
        return 2
    elif row['cell_fate.2']:
        return 3
    elif row['cell_fate.3']:
        return 4
    elif row['cell_fate.4']:
        return 5
    elif row['cell_fate.5']:
        return 6
    elif row['cell_fate.6']:
        return 7
    elif row['cell_fate.7']:
        return 8
    elif row['cell_fate.8']:
        return 9
    elif row['cell_fate.9']:
        return 10
    elif row['cell_fate.10']:
        return 11
    elif row['cell_fate.11']:
        return 12
    elif row['cell_fate.12']:
        return 13
    else:
        return 0 

# create a new column 13 different cell fates
branch_spot_table['cell_fate'] = branch_spot_table.apply(cell_fate_values_to_label, axis=1)

### Remove unnecessary columns to save some RAM

In [6]:
columns_to_keep = ['label', 'Branch Average Movement', 'Branch Movement Direction', 'Branch Movement Direction.1', 'Branch Movement Direction.2', 'Branch N Successors and Predecessors', 'Branch N Successors and Predecessors.1', 'Branch Sinuosity', 'Branch duration and displacement', 'Branch duration and displacement.1', 'Branch relative movement', 'Branch relative movement.1', 'Branch relative movement.2', 'Branch relative movement.3', 'Cell division frequency' ]
if 'cell_fate' in branch_spot_table.columns:
    columns_to_keep.append('cell_fate')
branch_spot_table = branch_spot_table[columns_to_keep]
branch_spot_table.head(2)


,label,Branch Average Movement,Branch Movement Direction,Branch Movement Direction.1,Branch Movement Direction.2,Branch N Successors and Predecessors,Branch N Successors and Predecessors.1,Branch Sinuosity,Branch duration and displacement,Branch duration and displacement.1,Branch relative movement,Branch relative movement.1,Branch relative movement.2,Branch relative movement.3,Cell division frequency
0,1,0.845989,0.888329,0.458005,-0.033209,2,0,9.896631,24.106064,282.0,-0.794458,0.606264,0.035788,0.891979,0.001379
1,2,0.785019,-0.840221,0.491131,-0.229825,2,0,11.412242,12.037811,175.0,-0.972409,0.230314,-0.037106,0.952798,0.001202


### Rename columns to have more meaningful names

In [7]:
new_columns = {'Branch duration and displacement.1': 'Branch duration', 'Branch duration and displacement': 'Branch displacement', 'Branch Movement Direction' : 'Branch Movement Direction (X)', 'Branch Movement Direction.1' : 'Branch Movement Direction (Y)', 'Branch Movement Direction.2' : 'Branch Movement Direction (Z)', 'Branch relative movement' : 'Branch Relative Movement (x-component)', 'Branch relative movement.1' : 'Branch Relative Movement (y-component)', 'Branch relative movement.2' : 'Branch Relative Movement (z-component)', 'Branch relative movement.3' : 'Branch Relative Movement', 'Branch N Successors and Predecessors' : 'Branch N Successors', 'Branch N Successors and Predecessors.1' : 'Branch N Predecessors'}

if 'cell_fate' in branch_spot_table.columns:
    new_columns['cell_fate'] = 'Cell fate_CLUSTER_ID'

# Rename the columns using the dictionary
branch_spot_table.rename(columns=new_columns, inplace=True)
branch_spot_table.head(2)

,label,Branch Average Movement,Branch Movement Direction (X),Branch Movement Direction (Y),Branch Movement Direction (Z),Branch N Successors,Branch N Predecessors,Branch Sinuosity,Branch displacement,Branch duration,Branch Relative Movement (x-component),Branch Relative Movement (y-component),Branch Relative Movement (z-component),Branch Relative Movement,Cell division frequency
0,1,0.845989,0.888329,0.458005,-0.033209,2,0,9.896631,24.106064,282.0,-0.794458,0.606264,0.035788,0.891979,0.001379
1,2,0.785019,-0.840221,0.491131,-0.229825,2,0,11.412242,12.037811,175.0,-0.972409,0.230314,-0.037106,0.952798,0.001202


### Optionally export measurements to CSV file
This can be skipped if the measurements are not needed outside napari.

In [8]:
filename = 'measurements_branch_spot.csv'
branch_spot_table.to_csv(tables_folder_path + '/' + filename, sep=',', quotechar='"', index=False)

## View in napari
* Installation instructions for napari can be found [here](https://biapol.github.io/blog/mara_lampert/getting_started_with_mambaforge_and_python/readme.html).

### Read label image
The label image is expected to be exported from Mastodon with the following settings:
* Label Id: *Branch spot ID*
* Frame rate reduction: expected to be the same as the frame reduction factor specified above

#### Set path to label image
Enter the path to the label image exported from Mastodon here.

In [9]:
label_image_path = ('')

#### Read label image from path

In [10]:
#### Read label image from path
label_image_path = Path(label_image_path)
label_image = imread(label_image_path)


#### Optionally print shape of label image to check if everything is ok, order: t, z, y, x

In [11]:
print(label_image.shape)

(504, 12, 500, 1024)


### Optionally read intensity image
This will only work, if the intensity image has the same frame reduction rate as the label image.

#### Optionally Set path to intensity image
Enter the path to the intensity image here.

In [35]:
intensity_image_path = ''

#### Optionally read intensity image from path

In [36]:
intensity_image_path = Path(intensity_image_path)
intensity_image = imread(intensity_image_path)

#### Optionally print shape of intensity image to check if everything is ok. Order: t, z, y, x

In [37]:
print(intensity_image.shape)

(51, 12, 500, 1024)


### Open napari viewer

In [12]:
viewer = napari.Viewer()

### Add label image

In [13]:
labels_layer = viewer.add_labels(label_image, features=branch_spot_table)

### Set scale of label image
Due to bugs both in Mastodon export and in Napari import scale needs to be set manually.
Expected order: t, z, y, x

In [14]:
# labels_layer.scale = (1, 2.48, 0.31196313094933187, 0.31196313094933187)
# labels_layer.scale = (1, 2.03, 0.41, 0.41)
# set scale in napari terminal
# viewer.layers[0].scale = (1, 2.48, 0.31196313094933187, 0.31196313094933187)

### Optionally add intensity image
This will only work, if the intensity image has the same frame reduction rate as the label image.

In [ ]:
intensity_layer = viewer.add_image(intensity_image)

### Optionally set scale of intensity image
Due to bugs both in Mastodon export and in Napari import scale needs to be set manually.
Expected order: t, z, y, x
Should be the same as for the label image.

In [ ]:
# intensity_layer.scale = (1, 2.48, 0.31196313094933187, 0.31196313094933187)
# intensity_layer.scale = (1, 2.03, 0.41, 0.41)

### Turn on 3D view

In [15]:
viewer.dims.ndisplay = 3

### Load napari-clusters-plotter plugin

In [16]:
viewer.window.add_plugin_dock_widget(plugin_name='napari-clusters-plotter', widget_name='Plotter Widget')


(<napari._qt.widgets.qt_viewer_dock_widget.QtViewerDockWidget at 0x25120b8d3a0>,
 <napari_clusters_plotter._plotter.PlotterWidget at 0x251206f7430>)

## You are ready to use the napari-clusters-plotter with the Mastodon data plugin now.
Consult the documentation of the napari-clusters-plotter for further instructions, if needed.
* https://www.youtube.com/watch?v=qZ8KDrgL1Ro
* https://github.com/BiAPoL/napari-clusters-plotter